#tag mission

### In the process of fine-tuning the LLM, we needed to provide the machine with 2,000 to 10,000 tagged examples. Due to the complexity of the task and the fact that HeBERT is primarily designed for sentiment analysis, we determined that 6,000 examples would provide a solid foundation for the model. Additionally, throughout the process, we encountered a challenging parsing issue. To address this, we utilized an API for part of the tagging task, while the remaining tags were applied manually by us

###upload relevent packages

In [ ]:
!pip install openai==0.28
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.7 MB/s eta 0:00:00


In [ ]:
import openai
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import os
import json
import tiktoken

In [ ]:
# The name of the file later changed to labeled1.xlsx
df = pd.read_csv('/content/train (1).csv')
df = df.drop(columns=["Unnamed: 0"])
df = df[df['text'].str.len() < 5000]
df.to_csv('train_data_under_5000_tokens_per_speech.csv',index=False)
df.shape

(8993, 1)

In [ ]:
df

,text
0,סעיף קטן (ד) מאפשר לגבות את הסכום הזה.סעיף קטן...
1,בסדר.נרשום שאתם מגישים הסתייגות.
2,שביקשו ממנו לעשות פעולה ברכוש.
3,"אני מסכים, אבל אני לא יכול להשאיר לשר את הדיון..."
4,אז מה שקרה לילדה הזאת – תשווי את זה לקטיעת רגל- -
...,...
8995,יש כאן צורך לקבוע את דרכי ההכרזה.דרכי ההכרזה כ...
8996,במהלך ההפסקה ניסינו יצר מין תהך שיכול לאחד את ...
8997,"קודם כול, ברשותכם, אני אפתח בזה שאנחנו נמצאים ..."
8998,רק על המחסומים הפנימיים.לא בין השטחים למדינת י...


In [ ]:
openai.api_key = 'confidential'

In [ ]:
def chat_with_gpt(prompt):
  response = openai.ChatCompletion.create(
      model="gpt-4",
      messages=[
          {"role": "system", "content": "You are the best nlp model assistant."},
          {"role": "user", "content": f"""
            Please rate the degree of offensiveness of the following list of sentences.
            Provide a score between 0 and 10, where 0 means the sentence is not offensive at all, and 10 means it is extremely offensive.
            Consider the offensiveness against the legal system, including judges, lawyers, and legal consultants, as well as general offensiveness.
            Ensure consistency in your ratings, and distinguish between legitimate criticism and offensive statements.
            The output must be in JSON format, listing only the index and score for each sentence without returning the sentence itself.
            For example, if there are two sentences, the output should look like: {{"0":2, "1":7}} and nothing more, no explains, just the json.
            Variance in the scores is important, with at least 3-4 different values in total
            {prompt}.
            """}
      ]
  )
  response = response.choices[0].message.content
  return response



'{"0":7}'

In [ ]:
prompt_string = f"""
            Please rate the degree of offensiveness of the following list of sentences.
            Provide a score between 0 and 10, where 0 means the sentence is not offensive at all, and 10 means it is extremely offensive.
            Consider the offensiveness against the legal system, including judges, lawyers, and legal consultants, as well as general offensiveness.
            Ensure consistency in your ratings, and distinguish between legitimate criticism and offensive statements.
            The output must be in JSON format, listing only the index and score for each sentence without returning the sentence itself.
            For example, if there are two sentences, the output should look like: {{"0":2, "1":7}} and nothing more, no explains, just the json.
            Variance in the scores is important, with at least 3-4 different values in total"""

prompt_len = len(prompt_string.split())
prompt_len

124

In [ ]:
def divide_df(df, max_tokens=int(3800 - prompt_len), model_name="gpt-4"):
    encoder = tiktoken.encoding_for_model(model_name)
    current_list = []
    total_tokens = 0
    result = []
    for text in df['text']:
        token_count = len(encoder.encode(text))
        if total_tokens + token_count <= max_tokens:
            current_list.append(text)
            total_tokens += token_count
        else:
            result.append(current_list)
            current_list = [text]
            total_tokens = token_count
    if current_list:
        result.append(current_list)
    return result

list_for_gpt = divide_df(df)
len(list_for_gpt)

403

## create a tagged learning table

In [ ]:
list_for_gpt = divide_df(df)
df_scores = pd.DataFrame(columns=['score'])
list_of_scores = []
output_list = []

for i in range(len(list_for_gpt)):
  ouput_sub_list = []
  with open('tagged_data_to_parse_later.txt',"a+") as f:
    try:
      f.write(f"len of chunk {i}: {len(list_for_gpt[i])}"+ f"iteration {i} output:")
      f.write("\n")
      print(i)
      json = chat_with_gpt(prompt = list_for_gpt[i])
      output_list.append(list(zip(list_for_gpt[i], json)))
      f.write(json)
      f.write(f"finish successfully iter {i}")
      f.write("\n")

    except Exception as e:
      print(f"an error occured in iteration: {i}")
      f.write(f"exception occured in iteration {i}")
      print(str(e))
      f.write(f"{str(e)}\n")


0
an error occured in iteration: 0
You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.
1
an error occured in iteration: 1
You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.
2
an error occured in iteration: 2
You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.
3
an error occured in iteration: 3
You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.
4
an error occured in iteration: 4
You exceeded your current quota, please check your plan a

KeyboardInterrupt: 

#### As seen above, we have reached the $50 limit for API usage, so from this point onward, we will proceed with manual tagging.